---

# Model deployment

## Importar librerias

In [1]:
import mlflow
import sys
import os

sys.path.append("../../")
from ML.utils.mlflow_flow import set_tracking
from dotenv import load_dotenv

load_dotenv()
ENDPOINT_URL = os.getenv("ENDPOINT_URL")
DAGSHUB_PUBLIC_ACCESS_KEY = os.getenv("MLFLOW_TRACKING_TOKEN")

os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow"
os.environ["MLFLOW_REGISTRY_URI"] = "https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow"



---

## Descargar modelo

In [2]:
from mlflow.tracking import MlflowClient

sys.path.append("../models/ensemble_elnet_lgbm")

MODEL_NAME = 'elnet_lgbm'


print("TRACKING:", os.getenv("MLFLOW_TRACKING_URI"))
print("REGISTRY:", os.getenv("MLFLOW_REGISTRY_URI"))
set_tracking(ENDPOINT_URL)

c = MlflowClient()
mv = c.get_model_version_by_alias("elnet_lgbm", "champion")
print("Alias -> version:", mv.version, "stage:", mv.current_stage)


TRACKING: https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow
REGISTRY: https://dagshub.com/yosesotomayor/retoCasas_v2.mlflow


MlflowException: API request to endpoint /api/2.0/mlflow/registered-models/alias failed with error code 401 != 200. Response body: ''

---